In [1]:
PREFIX="https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL="${PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL="${PREFIX}/hair_classifier_v1.onnx"

In [2]:
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data

--2025-12-08 17:12:10--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
Resolving github.com (github.com)... 20.26.156.215
Connecting to github.com (github.com)|20.26.156.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/398ded4a-c41c-4e5a-9672-acb7e441de54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-08T17%3A52%3A53Z&rscd=attachment%3B+filename%3Dhair_classifier_v1.onnx.data&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-08T16%3A52%3A05Z&ske=2025-12-08T17%3A52%3A53Z&sks=b&skv=2018-11-09&sig=UuNhVhiIaVzNLejVqgnSlIdTqoHlD33fItWsYRwcUU4%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTIxNTczMCwibmJmIjoxNzY1MjEzOTMwLCJwY

In [3]:
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx

--2025-12-08 17:12:29--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx
Resolving github.com (github.com)... 20.26.156.215
Connecting to github.com (github.com)|20.26.156.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/c6b83ad5-a901-40e9-bf2c-41ad174c870c?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-08T17%3A54%3A01Z&rscd=attachment%3B+filename%3Dhair_classifier_v1.onnx&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-08T16%3A53%3A26Z&ske=2025-12-08T17%3A54%3A01Z&sks=b&skv=2018-11-09&sig=Apq%2FENjIv2ijUEc3RpNHP%2FehhWbESrxLx8Z5xer2%2BFc%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTIxNDI0OSwibmJmIjoxNzY1MjEzOTQ5LCJwYXRoI

In [1]:
!pip install torchvision

In [2]:
import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image
import torch
from torchvision import transforms

In [3]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size=(200, 200)):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [5]:
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
img = download_image(url)
img = prepare_image(img)

img_np = np.array(img).astype("float32") / 255.0

preprocess = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

img_tensor = torch.tensor(img_np).permute(2, 0, 1)   
img_tensor = preprocess(img_tensor)

first_pixel_R = img_tensor[0, 0, 0].item()
first_pixel_R

-1.0732940435409546

In [7]:
import onnxruntime as ort

ort_session = ort.InferenceSession("hair_classifier_v1.onnx")

input_tensor = img_tensor.unsqueeze(0).numpy().astype(np.float32) 

input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name

pred_onx = ort_session.run([output_name], {input_name: input_tensor})
output_value = pred_onx[0][0][0] 

output_value

np.float32(0.09156641)